In [1]:
## This repo: https://github.com/ReagentX/purple_air_api/

import pandas as pd
import purpleair as pa
import datetime as dt

In [2]:
from purpleair.network import SensorList
p = SensorList()

Initialized 22,230 sensors!


In [3]:
df = p.to_dataframe(sensor_filter='outside', # or 'all'
                    channel='parent')

df.columns
df.head()
df.shape

(16360, 43)

In [4]:
## Save IDs of PA sensors that are outdoors:
df.to_csv("C:\\Users\\ellen\\OneDrive\\MyDocs\\Graduate Research\\Low-cost AQ sensor epi\\Getting data\\PA_outside.csv")

In [3]:
## Getting sensors in CA that are collocated with (within 50 meters of) AQS monitors:
CA_locs = pd.read_csv("C:\\Users\\ellen\\OneDrive\\MyDocs\\Graduate Research\\Low-cost AQ sensor epi\\Getting data\\NN_PA_AQS_50m-outdoor.csv")
CA_locs.head()

,Lon,Lat,Site.ID,Parameter,PA.ID,Dist
0,-119.773210,36.785380,11,88101,6568,0.021083
1,-115.539170,32.978330,7,88101,77269,0.016803
2,-118.330783,37.360684,2,88101,17107,0.007901
3,-117.871036,36.487823,1003,88101,13469,0.028659
4,-119.062613,35.356615,14,88101,2350,0.000620


In [10]:
ids = list(set(CA_locs["PA.ID"]))
len(ids)

35

In [3]:
import inspect

se = pa.sensor.Sensor(61200)

se.parent.get_historical.__code__.co_varnames

Child sensor requested, acquiring parent instead.


('self',
 'weeks_to_get',
 'thingspeak_field',
 'start_date',
 'channel',
 'key',
 'parent_cols',
 'child_cols',
 'columns',
 'to_week',
 'url',
 'weekly_data',
 '_')

In [22]:
## Testing for one sensor
## Note: need to run this to get the column names for the full run

start = dt.datetime.now()

se = pa.sensor.Sensor(61200)

# se.parent = channel A, se.child = channel B
h = se.parent.get_historical(weeks_to_get=5,
                            start_date = dt.datetime.strptime("2021-01-01", "%Y-%m-%d"),
                             thingspeak_field='primary')
end = dt.datetime.now()

dt.timedelta.total_seconds(end-start) # 1 week = 1.247s, 5 weeks = 7.540s

h.columns


Child sensor requested, acquiring parent instead.


Index(['created_at', 'PM1.0 (CF=1) ug/m3', 'PM2.5 (CF=1) ug/m3',
       'PM10.0 (CF=1) ug/m3', 'UptimeMinutes', 'ADC', 'Temperature_F',
       'Humidity_%', 'PM2.5 (CF=ATM) ug/m3'],
      dtype='object')

In [25]:
## Get all data from 2020, from the nearest neighbor LCS to the AQS (within 50m).
data = pd.DataFrame(columns = h.columns)
data["ID"] = []

for s in ids[10:35]: # sensor 9 didn't have data, so had to restart in the middle
    se = pa.sensor.Sensor(s)
    print(ids.index(s))
    
    # se.parent = channel A, se.child = channel B
    h = se.child.get_historical(weeks_to_get=53,
                                start_date = dt.datetime.strptime("2021-01-01", "%Y-%m-%d"), 
                                 thingspeak_field='primary') 
    h["ID"] = s
    data = data.append(h)

## Change name of output file for channels A and B:
data.to_csv("C:\\Users\\ellen\\OneDrive\\MyDocs\\Graduate Research\\Low-cost AQ sensor epi\\Getting data\\PA_2020_NNs_channelB_2.csv")

10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34


In [4]:
data = pd.read_csv("C:\\Users\\ellen\\OneDrive\\MyDocs\\Graduate Research\\Low-cost AQ sensor epi\\Getting data\\PA_2020_NNs_channelB_2.csv")
data.head()
data.tail()

,Unnamed: 0,created_at,PM1.0 (CF=1) ug/m3,PM2.5 (CF=1) ug/m3,PM10.0 (CF=1) ug/m3,UptimeMinutes,RSSI_dbm,Atmospheric Pressure,gas_sensor,PM2.5 (CF=ATM) ug/m3,ID
4827629,347843,2019-12-26 23:50:47,0.0,0.0,0.00,19048.0,0.0,1007.19,NaN,0.0,9594.0
4827630,347844,2019-12-26 23:52:47,0.0,0.0,0.00,19048.0,0.0,1007.25,NaN,0.0,9594.0
4827631,347845,2019-12-26 23:54:47,0.0,0.0,0.04,19560.0,0.0,1007.27,NaN,0.0,9594.0
4827632,347846,2019-12-26 23:56:47,0.0,0.0,0.17,19560.0,0.0,1007.26,NaN,0.0,9594.0
4827633,347847,2019-12-26 23:58:47,0.0,0.0,0.09,19560.0,0.0,1007.26,NaN,0.0,9594.0
